# Homework №3

### <center>Student: Puzyrev Dmitri</center>

# Description

1. Those how are making projects on artilces: half of you need to choose co-authorship graph, others - citation (but consider citation graph as undirected for simplicity)
2. Use weighted graph so that it would give you ability to calculate embeddings in more appropriate way

### Guidelines:

1. Initiallize your classification set as follows:
    * Determine training and testing intervals on your time domain (for example, take a period $2000$-$2014$ as training period and $2015$-$2018$ as testing period)
    * Pick pairs of nodes that **have appeared during training interval** but **had no links** during it
    * These pairs form **positive** or **negative** examples depending on whether they have formed coauthorships **during the testing interval**
    * You have arrived to binary classification problem.
2. Construct feature space:
    * Use at least 2 features based on neighborhood 
    * Use at least 2 fetures based on shortest path
    * Use embedding representation of nodes' pairs (for example, node2vec)
    * Use idea of time series features (with time lag)
    * Use idea of change-point detection
3. Choose at least $3$ classification algorithms and compare them in terms of Accuracy, Precision, Recall, F-Score (for positive class) and Mean Squared Error. Use k-fold cross-validation and average your results

In [1]:
import mysql.connector
import networkx as nx
import numpy as np
from node2vec import Node2Vec
from node2vec.edges import HadamardEmbedder
from tqdm import tqdm

First, we have to retreive the needed graph elements. For that, we appeal to our project SQL database and get entries for desired years.

In [2]:
mydb = mysql.connector.connect(
  host="articlesgrap.cxqhtfp4sprs.eu-central-1.rds.amazonaws.com",
  user="wizard",
  passwd="12345678",
  database="TheData"
)


def getAutGraph(by, fy):
    '''get co-authors between years by and fy'''
    mycursor = mydb.cursor()
    sql = "SELECT a.auid, b.auid, count(b.auid) as weight FROM TheData.isAuthor as a join TheData.isAuthor as b on a.artid=b.artid join TheData.Articles as c on a.artid=c.id where c.year between %s and %s and a.auid <> b.auid group by a.auid, b.auid order by a.auid desc;"
    val = (by, fy)
    mycursor.execute(sql, val)
    myresult = mycursor.fetchall()
    mycursor.close()
    return myresult

Let's consider years 2014-2015 as <b>training</b> interval and 2016-2017 as <b>testing</b> interval. We also intend to implement time series lagging, that's why we also need to get year 2013 for 2014.

In [3]:
aut_2013 = getAutGraph(2013,2013)
G2013 = nx.Graph()
G2013.add_weighted_edges_from(aut_2013)

print('2013')

aut_2014 = getAutGraph(2014,2014)
G2014 = nx.Graph()
G2014.add_weighted_edges_from(aut_2014)

print('2014')

aut_2015 = getAutGraph(2015,2015)
G2015 = nx.Graph()
G2015.add_weighted_edges_from(aut_2015)

print('2015')

aut_test = getAutGraph(2016,2017)
Gtest = nx.Graph()
Gtest.add_weighted_edges_from(aut_test)

print('test')

2013
2014
2015
test


Now, we need to select features for classification. First, let's strip components which are absent in testing interval. Next, Select edges which are not connected in train but are present in both train and test. These are objects for our classification.

We also select edges present in previous year as 'time series' factor for current connections

In [4]:
def filter_test(train, test):
    common = train.nodes & test.nodes
    
    components = nx.connected_component_subgraphs(train)
    G = nx.Graph()
    for component in components:
        if common & component.nodes:
            G = nx.compose(G, component)

    no_connect = list(nx.non_edges(G))
    
    suitable_edges = [edge for edge in no_connect if edge[0] in common and edge[1] in common]

    return G, suitable_edges
    
def filter_lag(lag, train, suitable_edges):
    common = lag.nodes & train.nodes
    
    lag_edges = [edge for edge in suitable_edges if edge[0] in common and edge[1] in common]
    
    return lag_edges

We consider the two following neighbourhood metrics: Jaccard coefficient and resource allocation index.

In [5]:
def neighbourhood_metrics(G, edges):
    jaccard = nx.jaccard_coefficient(G, edges)
    res_alloc = nx.resource_allocation_index(G, edges)
    return jaccard, res_alloc

For distance metrics, shortest path (measured by Dijkstra algorithm) and vertex eccentricity are considered.

In [6]:
def distance_metrics(G):
    dijkstra = dict(nx.all_pairs_dijkstra_path_length(G))
    components = nx.connected_component_subgraphs(G)
    eccentricity = []
    for comp in components:
        eccentricity.append(nx.eccentricity(comp))
    return dijkstra, eccentricity

Here, 20-dimension node2vec embeddings are created.

In [7]:
def construct_embeddings(G):
    node2vec = Node2Vec(G, dimensions=20, walk_length=50, num_walks=200, workers=8)
    model = node2vec.fit(window=10, min_count=1, batch_words=4, workers=8)
    print('node2vec model trained')
    edges_embs = HadamardEmbedder(keyed_vectors=model.wv)
    print('embeds ready')
    
    return edges_embs

Now we are ready to assemble training data. After filtration, neighbourhood metrics are counted for both previous and current points in time series. For eccentricity, the feature factor is the sum of eccentricities of the corresponding nodes. We also mark the label for each edge if it is not present in test set, then label is set to False.

In [8]:
def get_features(Glag, Gtrain, Gtest):
    Gfiltered, authors = filter_test(Gtrain, Gtest)
    lags = filter_lag(Glag, Gtrain, authors)
    
    jac, resalc = neighbourhood_metrics(Gfiltered, authors)
    jac_lag, resalc_lag = neighbourhood_metrics(Glag, lags)
    
    path, ecc = distance_metrics(Gfiltered)
    
    emb = construct_embeddings(Gfiltered)
    
    jaclag = next(jac_lag)
    resalclag = next(resalc_lag)
    
    data = []
    for j, r, n in tqdm(zip(jac, resalc, authors)):
        a = n[0]
        b = n[1]
        if (b in path[a]):
            d = path[a][b]
        else:
            d = 999999999
        for comp in ecc:
            if a in comp:
                ea = comp[a]
            if b in comp:
                eb = comp[b]
        e = ea + eb
        if (jaclag != None and a == jaclag[0] and b == jaclag[1]):
            jl = jaclag
            try:
                jaclag = next(jac_lag)
            except:
                jaclag = None
        else:
            jl = (None, None, 0)
        if (resalclag != None and a == resalclag[0] and b == resalclag[1]):
            rl = resalclag
            try:
                resalclag = next(resalc_lag)
            except:
                resalclag = None
        else:
            rl = (None, None, 0)
        
        label = b in Gtest[a]
        data.append([a, b, j[2], r[2], jl[2], rl[2], d, e] + list(emb[(str(a), str(b))]) + [label])
    print(len(data))
    return data

We need to get features for both years of training set with corresponding lags. By concatenating them, we will have the complete dataset.

In [9]:
%%time

examples2014 = get_features(G2013, G2014, Gtest)

Computing transition probabilities: 100%|██████████| 1000/1000 [00:25<00:00, 39.74it/s]
1038it [00:00, 10373.46it/s]

node2vec model trained
embeds ready


20122it [00:02, 8866.14it/s]

20122
CPU times: user 3min 54s, sys: 24.4 s, total: 4min 18s
Wall time: 4min


In [10]:
%%time

examples2015 = get_features(G2014, G2015, Gtest)

Computing transition probabilities: 100%|██████████| 1576/1576 [00:05<00:00, 272.07it/s]
845it [00:00, 8445.66it/s]

node2vec model trained
embeds ready


55761it [00:06, 8786.85it/s]

55761
CPU times: user 5min 10s, sys: 38.6 s, total: 5min 49s
Wall time: 5min 25s


In [11]:
data = np.array(examples2014 + examples2015)

print(data)

[[ 1.84381522e+10  7.00519835e+09  0.00000000e+00 ... -1.74617136e+00
  -7.63035536e-01  0.00000000e+00]
 [ 1.84381522e+10  7.00445493e+09  0.00000000e+00 ...  3.14624637e-01
   9.11662757e-01  0.00000000e+00]
 [ 1.84381522e+10  6.60396034e+09  0.00000000e+00 ...  9.61430147e-02
   2.65502214e+00  0.00000000e+00]
 ...
 [ 5.57571276e+10  5.51143178e+10  0.00000000e+00 ...  6.37804806e-01
  -5.07885647e+00  0.00000000e+00]
 [ 5.57571276e+10  1.64815339e+10  0.00000000e+00 ... -1.57295489e+00
  -3.00056982e+00  0.00000000e+00]
 [ 1.64815339e+10  5.51143178e+10  0.00000000e+00 ... -6.02263272e-01
   1.61391389e+00  0.00000000e+00]]


In [17]:
X, y = data[:,2:-1], data[:,-1]

print(X, y)

[[ 0.          0.          0.         ...  0.73601514 -1.74617136
  -0.76303554]
 [ 0.          0.          0.         ...  0.15961549  0.31462464
   0.91166276]
 [ 0.          0.          0.         ...  0.76863754  0.09614301
   2.65502214]
 ...
 [ 0.          0.          0.         ...  5.06736231  0.63780481
  -5.07885647]
 [ 0.          0.          0.         ... -0.7280708  -1.57295489
  -3.00056982]
 [ 0.          0.          0.         ... -0.58295172 -0.60226327
   1.61391389]] [0. 0. 0. ... 0. 0. 0.]


It is worth noting that only small proportion of edges get positive class.

In [18]:
print(np.sum(y == 0))
print(np.sum(y == 1))

74283
1600


Let's divide our set to train and test in proportions of 75 to 25%

In [22]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

print(X_train.shape)
print(X_test.shape)

print(np.sum(y_test == 1))

(56912, 26)
(18971, 26)
383


This is for parameter tuning

In [20]:
from sklearn.model_selection import GridSearchCV

def tune_model(model, params, train, target):
    clf = GridSearchCV(model, params, cv=3, verbose=2, n_jobs=-1)
    clf.fit(train, target)
    print(clf.best_estimator_)
    print(clf.best_score_)

First, we can look how the simpliest classification algorithm - logistic regression - will perform.

In [21]:
from sklearn.linear_model import LogisticRegression

logreg = LogisticRegression()

logreg.fit(X_train, y_train)

pred = logreg.predict(X_test)

print(accuracy_score(y_test, pred))
print(precision_score(y_test, pred))
print(recall_score(y_test, pred))
print(f1_score(y_test, pred))

0.9798112909177165
0.0
0.0
0.0


/home/willstudent/.local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/willstudent/.local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


Next, we test Random Forest Classification model, preemptively picking optimal parameters.

In [31]:
from sklearn.ensemble import RandomForestClassifier

forest = RandomForestClassifier(random_state=42)
params = {'n_estimators': [10, 20, 50, 100, 200], 'max_depth': [2, 5, 10, 50, None], 'min_samples_split': [2, 5, 10], 
          'min_samples_leaf': [1, 2, 5, 10, 50]}

tune_model(forest, params, X_train, y_train)

Fitting 3 folds for each of 375 candidates, totalling 1125 fits
[CV] max_depth=2, min_samples_leaf=1, min_samples_split=2, n_estimators=20 
[CV] max_depth=2, min_samples_leaf=1, min_samples_split=2, n_estimators=20 
[CV] max_depth=2, min_samples_leaf=1, min_samples_split=2, n_estimators=10 
[CV] max_depth=2, min_samples_leaf=1, min_samples_split=2, n_estimators=10 
[CV] max_depth=2, min_samples_leaf=1, min_samples_split=2, n_estimators=10 
[CV] max_depth=2, min_samples_leaf=1, min_samples_split=2, n_estimators=20 
[CV] max_depth=2, min_samples_leaf=1, min_samples_split=2, n_estimators=50 
[CV] max_depth=2, min_samples_leaf=1, min_samples_split=2, n_estimators=50 
[CV]  max_depth=2, min_samples_leaf=1, min_samples_split=2, n_estimators=10, total=   0.5s
[CV]  max_depth=2, min_samples_leaf=1, min_samples_split=2, n_estimators=10, total=   0.6s
[CV] max_depth=2, min_samples_leaf=1, min_samples_split=2, n_estimators=50 
[CV] max_depth=2, min_samples_leaf=1, min_samples_split=2, n_estimator

[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   12.3s


[CV]  max_depth=2, min_samples_leaf=1, min_samples_split=10, n_estimators=10, total=   0.7s
[CV] max_depth=2, min_samples_leaf=1, min_samples_split=10, n_estimators=20 
[CV]  max_depth=2, min_samples_leaf=1, min_samples_split=10, n_estimators=10, total=   0.5s
[CV] max_depth=2, min_samples_leaf=1, min_samples_split=10, n_estimators=20 
[CV]  max_depth=2, min_samples_leaf=1, min_samples_split=10, n_estimators=10, total=   0.6s
[CV] max_depth=2, min_samples_leaf=1, min_samples_split=10, n_estimators=20 
[CV]  max_depth=2, min_samples_leaf=1, min_samples_split=5, n_estimators=100, total=   5.0s
[CV] max_depth=2, min_samples_leaf=1, min_samples_split=10, n_estimators=50 
[CV]  max_depth=2, min_samples_leaf=1, min_samples_split=2, n_estimators=200, total=  10.0s
[CV] max_depth=2, min_samples_leaf=1, min_samples_split=10, n_estimators=50 
[CV]  max_depth=2, min_samples_leaf=1, min_samples_split=10, n_estimators=20, total=   1.1s
[CV] max_depth=2, min_samples_leaf=1, min_samples_split=10, n_e

[CV] max_depth=2, min_samples_leaf=2, min_samples_split=10, n_estimators=50 
[CV] max_depth=2, min_samples_leaf=2, min_samples_split=10, n_estimators=50 
[CV]  max_depth=2, min_samples_leaf=2, min_samples_split=5, n_estimators=100, total=   5.1s
[CV] max_depth=2, min_samples_leaf=2, min_samples_split=10, n_estimators=50 
[CV]  max_depth=2, min_samples_leaf=2, min_samples_split=10, n_estimators=20, total=   1.1s
[CV] max_depth=2, min_samples_leaf=2, min_samples_split=10, n_estimators=100 
[CV]  max_depth=2, min_samples_leaf=2, min_samples_split=2, n_estimators=200, total=  10.4s
[CV] max_depth=2, min_samples_leaf=2, min_samples_split=10, n_estimators=100 
[CV]  max_depth=2, min_samples_leaf=2, min_samples_split=10, n_estimators=50, total=   2.5s
[CV]  max_depth=2, min_samples_leaf=2, min_samples_split=10, n_estimators=50, total=   2.5s
[CV] max_depth=2, min_samples_leaf=2, min_samples_split=10, n_estimators=100 
[CV] max_depth=2, min_samples_leaf=2, min_samples_split=10, n_estimators=20

[CV]  max_depth=2, min_samples_leaf=5, min_samples_split=10, n_estimators=50, total=   2.8s
[CV] max_depth=2, min_samples_leaf=5, min_samples_split=10, n_estimators=100 
[CV]  max_depth=2, min_samples_leaf=5, min_samples_split=10, n_estimators=50, total=   2.6s
[CV] max_depth=2, min_samples_leaf=5, min_samples_split=10, n_estimators=200 
[CV]  max_depth=2, min_samples_leaf=5, min_samples_split=10, n_estimators=50, total=   2.8s
[CV] max_depth=2, min_samples_leaf=5, min_samples_split=10, n_estimators=200 
[CV]  max_depth=2, min_samples_leaf=5, min_samples_split=10, n_estimators=100, total=   5.0s
[CV] max_depth=2, min_samples_leaf=5, min_samples_split=10, n_estimators=200 
[CV]  max_depth=2, min_samples_leaf=5, min_samples_split=5, n_estimators=200, total=  10.3s
[CV] max_depth=2, min_samples_leaf=10, min_samples_split=2, n_estimators=10 
[CV]  max_depth=2, min_samples_leaf=5, min_samples_split=5, n_estimators=200, total=  10.5s
[CV] max_depth=2, min_samples_leaf=10, min_samples_split=2

...

[CV]  max_depth=None, min_samples_leaf=5, min_samples_split=2, n_estimators=100, total=  22.0s
[CV] max_depth=None, min_samples_leaf=5, min_samples_split=5, n_estimators=10 
[CV]  max_depth=None, min_samples_leaf=2, min_samples_split=10, n_estimators=200, total=  43.4s
[CV] max_depth=None, min_samples_leaf=5, min_samples_split=5, n_estimators=10 
[CV]  max_depth=None, min_samples_leaf=5, min_samples_split=5, n_estimators=10, total=   2.4s
[CV] max_depth=None, min_samples_leaf=5, min_samples_split=5, n_estimators=10 
[CV]  max_depth=None, min_samples_leaf=5, min_samples_split=5, n_estimators=10, total=   2.2s
[CV] max_depth=None, min_samples_leaf=5, min_samples_split=5, n_estimators=20 
[CV]  max_depth=None, min_samples_leaf=5, min_samples_split=2, n_estimators=100, total=  21.6s
[CV] max_depth=None, min_samples_leaf=5, min_samples_split=5, n_estimators=20 
[CV]  max_depth=None, min_samples_leaf=5, min_samples_split=5, n_estimators=10, total=   2.2s
[CV] max_depth=None, min_samples_leaf

[CV]  max_depth=None, min_samples_leaf=10, min_samples_split=5, n_estimators=10, total=   2.2s
[CV]  max_depth=None, min_samples_leaf=10, min_samples_split=2, n_estimators=100, total=  20.8s
[CV] max_depth=None, min_samples_leaf=10, min_samples_split=5, n_estimators=10 
[CV] max_depth=None, min_samples_leaf=10, min_samples_split=5, n_estimators=20 
[CV]  max_depth=None, min_samples_leaf=10, min_samples_split=5, n_estimators=10, total=   2.2s
[CV] max_depth=None, min_samples_leaf=10, min_samples_split=5, n_estimators=20 
[CV]  max_depth=None, min_samples_leaf=10, min_samples_split=5, n_estimators=10, total=   2.4s
[CV] max_depth=None, min_samples_leaf=10, min_samples_split=5, n_estimators=20 
[CV]  max_depth=None, min_samples_leaf=10, min_samples_split=2, n_estimators=100, total=  20.8s
[CV] max_depth=None, min_samples_leaf=10, min_samples_split=5, n_estimators=50 
[CV]  max_depth=None, min_samples_leaf=10, min_samples_split=5, n_estimators=20, total=   4.5s
[CV] max_depth=None, min_sam

[CV]  max_depth=None, min_samples_leaf=10, min_samples_split=10, n_estimators=200, total=  42.3s
[CV] max_depth=None, min_samples_leaf=50, min_samples_split=5, n_estimators=20 
[CV] max_depth=None, min_samples_leaf=50, min_samples_split=5, n_estimators=20 
[CV]  max_depth=None, min_samples_leaf=50, min_samples_split=5, n_estimators=10, total=   1.8s
[CV] max_depth=None, min_samples_leaf=50, min_samples_split=5, n_estimators=20 
[CV]  max_depth=None, min_samples_leaf=50, min_samples_split=2, n_estimators=100, total=  17.4s
[CV] max_depth=None, min_samples_leaf=50, min_samples_split=5, n_estimators=50 
[CV]  max_depth=None, min_samples_leaf=50, min_samples_split=5, n_estimators=20, total=   3.9s
[CV] max_depth=None, min_samples_leaf=50, min_samples_split=5, n_estimators=50 
[CV]  max_depth=None, min_samples_leaf=50, min_samples_split=5, n_estimators=20, total=   3.8s
[CV] max_depth=None, min_samples_leaf=50, min_samples_split=5, n_estimators=50 
[CV]  max_depth=None, min_samples_leaf=50,

[Parallel(n_jobs=-1)]: Done 1125 out of 1125 | elapsed: 28.5min finished


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=50, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=200, n_jobs=1,
            oob_score=False, random_state=42, verbose=0, warm_start=False)
0.9843091087995501


In [23]:
from sklearn.ensemble import RandomForestClassifier

forest = RandomForestClassifier(n_estimators=200, max_depth=50, random_state=42, n_jobs=-1)

forest.fit(X_train, y_train)

pred = forest.predict(X_test)

print(accuracy_score(y_test, pred))
print(precision_score(y_test, pred))
print(recall_score(y_test, pred))
print(f1_score(y_test, pred))

0.9849770702651416
0.788235294117647
0.34986945169712796
0.48462929475587696


The third model involves gradient boosting. We also tuned it before training.

In [43]:
from xgboost import XGBClassifier

boost = XGBClassifier(learning_rate=0.1, objective='binary:logistic', nthread=1, seed=42)

params = {'n_estimators': [200], 'max_depth': [50], 'min_child_weight': [2, 3], 'gamma': np.arange(0.1, 0.4, 0.1), 
          'subsample': np.arange(0.6, 0.9, 0.1), 'colsample_bytree': np.arange(0.6, 0.9, 0.1)}

tune_model(boost, params, X_train, y_train)

Fitting 3 folds for each of 128 candidates, totalling 384 fits
[CV] colsample_bytree=0.6, gamma=0.1, max_depth=50, min_child_weight=2, n_estimators=200, subsample=0.6 
[CV] colsample_bytree=0.6, gamma=0.1, max_depth=50, min_child_weight=2, n_estimators=200, subsample=0.6 
[CV] colsample_bytree=0.6, gamma=0.1, max_depth=50, min_child_weight=2, n_estimators=200, subsample=0.7 
[CV] colsample_bytree=0.6, gamma=0.1, max_depth=50, min_child_weight=2, n_estimators=200, subsample=0.6 
[CV] colsample_bytree=0.6, gamma=0.1, max_depth=50, min_child_weight=2, n_estimators=200, subsample=0.7 
[CV] colsample_bytree=0.6, gamma=0.1, max_depth=50, min_child_weight=2, n_estimators=200, subsample=0.7 
[CV] colsample_bytree=0.6, gamma=0.1, max_depth=50, min_child_weight=2, n_estimators=200, subsample=0.7999999999999999 
[CV] colsample_bytree=0.6, gamma=0.1, max_depth=50, min_child_weight=2, n_estimators=200, subsample=0.7999999999999999 


/home/willstudent/.local/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/willstudent/.local/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/willstudent/.local/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/willstudent/.local/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty ar

[CV]  colsample_bytree=0.6, gamma=0.1, max_depth=50, min_child_weight=2, n_estimators=200, subsample=0.6, total=  34.2s
[CV] colsample_bytree=0.6, gamma=0.1, max_depth=50, min_child_weight=2, n_estimators=200, subsample=0.7999999999999999 


/home/willstudent/.local/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


[CV]  colsample_bytree=0.6, gamma=0.1, max_depth=50, min_child_weight=2, n_estimators=200, subsample=0.6, total=  34.2s
[CV] colsample_bytree=0.6, gamma=0.1, max_depth=50, min_child_weight=2, n_estimators=200, subsample=0.8999999999999999 


...

[CV]  colsample_bytree=0.8999999999999999, gamma=0.4, max_depth=50, min_child_weight=3, n_estimators=200, subsample=0.8999999999999999, total=  45.2s


/home/willstudent/.local/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/willstudent/.local/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


[CV]  colsample_bytree=0.8999999999999999, gamma=0.4, max_depth=50, min_child_weight=3, n_estimators=200, subsample=0.8999999999999999, total=  43.8s


[Parallel(n_jobs=-1)]: Done 384 out of 384 | elapsed: 37.7min finished


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8999999999999999, gamma=0.30000000000000004,
       learning_rate=0.1, max_delta_step=0, max_depth=50,
       min_child_weight=2, missing=None, n_estimators=200, n_jobs=1,
       nthread=1, objective='binary:logistic', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, seed=42, silent=True,
       subsample=0.7999999999999999)
0.9862243463592916


In [ ]:
from xgboost import XGBClassifier

boost = XGBClassifier(n_estimators=200, max_depth=50, min_child_weight=2, gamma=0.3, 
                      subsample=0.8, colsample_bytree=0.9, seed=42, nthread=8)

boost.fit(X_train, y_train)

pred_xgb = boost.predict(X_test)

print(accuracy_score(y_test, pred_xgb))
print(precision_score(y_test, pred_xgb))
print(recall_score(y_test, pred_xgb))
print(f1_score(y_test, pred_xgb))